### Working with data

Pytorch has two ways to work with data
- torch.utils.data.Dataset
    - TorchText
    - TorchVision
    - TorchAudio 
- torch.utils.data.DataLoader

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Every TorchVision Dataset includes two arguments: `transform` and `target_transform` to modify the samples and labels

In [5]:
# Download training data from open datasets
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor() # 将图像数据从原来的像素范围(0 ~ 255) 归一化为0 到 1之间的浮点数值, 并转为Pytorch张量
)

# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

Pass the `Dataset` as an argument to `DataLoader`. 

In [7]:
batch_size = 64

# Create data loaders
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


### Creating Models

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [12]:
# Define Model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


#### Optimizing the Model Parameters  
  
To train a model, we need a **loss function** and an **optimizer**

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [19]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}] / {size:>5d}")

In [20]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100 * correct:>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [21]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n ---------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
 ---------------------
loss: 1.147435 [   64] / 60000
loss: 1.165379 [ 6464] / 60000
loss: 0.971115 [12864] / 60000
loss: 1.114397 [19264] / 60000
loss: 0.984085 [25664] / 60000
loss: 1.016999 [32064] / 60000
loss: 1.060883 [38464] / 60000
loss: 0.991088 [44864] / 60000
loss: 1.038761 [51264] / 60000
loss: 0.960916 [57664] / 60000
Test Error: 
 Accuracy: 65.8%, Avg loss: 0.981262 

Epoch 2
 ---------------------
loss: 1.030967 [   64] / 60000
loss: 1.069005 [ 6464] / 60000
loss: 0.857517 [12864] / 60000
loss: 1.023769 [19264] / 60000
loss: 0.897737 [25664] / 60000
loss: 0.924868 [32064] / 60000
loss: 0.987012 [38464] / 60000
loss: 0.915373 [44864] / 60000
loss: 0.964098 [51264] / 60000
loss: 0.895761 [57664] / 60000
Test Error: 
 Accuracy: 67.3%, Avg loss: 0.909829 

Epoch 3
 ---------------------
loss: 0.945320 [   64] / 60000
loss: 1.000981 [ 6464] / 60000
loss: 0.775426 [12864] / 60000
loss: 0.959381 [19264] / 60000
loss: 0.840503 [25664] / 60000
loss: 0.856640 [32064] / 600

#### Saving Models

In [24]:
torch.save(model.state_dict(), "chapter00_model/model.pth")
print("Saved Pytorch Model State to model.pth")

Saved Pytorch Model State to model.pth


#### Loading Models

In [25]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("chapter00_model/model.pth", weights_only=True))

<All keys matched successfully>

In [26]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: {predicted}, Actual: {actual}")

Predicted: Ankle boot, Actual: Ankle boot
